In [8]:
import polars as pl
import pandas as pd

In [ ]:
%%time

df_pl = pl.scan_csv('measurements.txt',schema_overrides = {'City':str, 'Temp':float}, separator=";")
df_pl.collect().head()

CPU times: user 1.3 ms, sys: 3.64 ms, total: 4.94 ms
Wall time: 5.7 ms


In [5]:
%%time
df_pl.group_by("City").agg(pl.mean("Temp").name.suffix("_mean"),pl.min("Temp").name.suffix("_min"), pl.max("Temp").name.suffix("_max")).collect()

CPU times: user 1min 35s, sys: 2min 53s, total: 4min 29s
Wall time: 5min 20s


City,Temp_mean,Temp_min,Temp_max
str,f64,f64,f64
"""Timbuktu""",27.995775,-27.1,79.5
"""Lahore""",24.300126,-26.1,74.4
"""Vientiane""",25.905933,-22.1,71.7
"""Sana'a""",20.005642,-29.1,70.6
"""Austin""",20.702303,-27.5,68.8
…,…,…,…
"""Thiès""",24.005168,-26.7,74.0
"""Upington""",20.400324,-26.5,70.1
"""Athens""",19.20125,-27.2,69.7


In [3]:
def calc_mean_min_max(df_pl: pl.LazyFrame):
    return (df_pl
            .group_by('City').agg(pl.mean("Temp").name.suffix("_mean"), 
                                  pl.min("Temp").name.suffix("_min"), 
                                  pl.max("Temp").name.suffix("_max")).lazy()
           )

In [7]:
%%time
df_pl.pipe(calc_mean_min_max).explain(optimized=True)

CPU times: user 855 μs, sys: 3.99 ms, total: 4.85 ms
Wall time: 17.7 ms


'AGGREGATE\n  [col("Temp").mean().alias("Temp_mean"), col("Temp").min().alias("Temp_min"), col("Temp").max().alias("Temp_max")] BY [col("City")]\n  FROM\n  simple π 2/2 ["Temp", "City"]\n    Csv SCAN [measurements.txt]\n    PROJECT 2/2 COLUMNS'

In [8]:
%%time
df_pl.pipe(calc_mean_min_max).collect().head()

CPU times: user 1min 36s, sys: 3min 46s, total: 5min 23s
Wall time: 5min 56s


City,Temp_mean,Temp_min,Temp_max
str,f64,f64,f64
"""Tirana""",15.210402,-33.3,62.8
"""Beijing""",12.900069,-37.7,64.6
"""Skopje""",12.407986,-37.5,63.5
"""Yinchuan""",9.006627,-39.3,57.6
"""Alexandria""",19.99194,-30.4,68.8


In [5]:
%%time
result= df_pl.pipe(calc_mean_min_max).collect()

CPU times: user 1min 38s, sys: 4min 24s, total: 6min 2s
Wall time: 6min 16s


In [14]:
result.write_csv("result.csv")